In [1]:
import pickle
import os

In [2]:
with open("/data/chodera/zhangi/perses_benchmark/neq/17/0/0_apo.pickle", "rb") as f:
    htf = pickle.load(f)

In [23]:
system = htf._topology_proposal.old_system

In [24]:
topology = htf._topology_proposal.old_topology

In [27]:
def get_ion_and_water_parameters(system, topology, positive_ion_name="NA", negative_ion_name="CL", water_name="HOH"):
    '''
    Get the charge, sigma, and epsilon for the positive and negative ions. Also get the charge of the water atoms.
    
    Parameters
    ----------
    system : simtk.openmm.System
        the system from which to retrieve parameters
    topology : app.Topology
        the topology corresponding to the above system from which to retrieve atom indices
    positive_ion_name : str, "NA"
        the residue name of each positive ion
    negative_ion_name : str, "CL"
        the residue name of each negative ion
    water_name : str, "HOH"
        the residue name of each water
    
    Returns
    -------
    pos_charge
        charge of positive ion
    pos_sigma
        sigma of positive ion
    pos_epsilon
        epsilon of positive ion
    neg_charge
        charge of negative ion
    neg_sigma
        sigma of negative ion
    neg_epsilon
        epsilon of negative ion
    O_charge
        charge of O atom in water
    H_charge
        charge of H atom in water
    
    '''
    
    # Get the indices
    pos_index = None
    neg_index = None
    O_index = None
    H_index = None
    for atom in topology.atoms():
        if atom.residue.name == positive_ion_name and not pos_index:
            pos_index = atom.index
        elif atom.residue.name == negative_ion_name and not neg_index:
            neg_index = atom.index
        elif atom.residue.name == water_name and (not O_index or not H_index):
            if atom.name == 'O':
                O_index = atom.index
            elif atom.name == 'H1':
                H_index = atom.index
    assert pos_index is not None, f"Error occurred when trying to turn a water into an ion: No positive ions with residue name {positive_ion_name} found"
    assert neg_index is not None, f"Error occurred when trying to turn a water into an ion: No negative ions with residue name {negative_ion_name} found"
    assert O_index is not None, f"Error occurred when trying to turn a water into an ion: No O atoms with residue name {water_name} and atom name O found"
    assert H_index is not None, f"Error occurred when trying to turn a water into an ion: No water atoms with residue name {water_name} and atom name H1 found" 
    
    # Get parameters from nonbonded force
    force_dict = {i.__class__.__name__: i for i in system.getForces()}
    if 'NonbondedForce' in [i for i in force_dict.keys()]:
        nbf = force_dict['NonbondedForce']
        pos_charge, pos_sigma, pos_epsilon = nbf.getParticleParameters(pos_index)
        neg_charge, neg_sigma, neg_epsilon = nbf.getParticleParameters(neg_index)
        O_charge, _, _ = nbf.getParticleParameters(O_index)
        H_charge, _, _ = nbf.getParticleParameters(H_index)
    
    return pos_charge, pos_sigma, pos_epsilon, neg_charge, neg_sigma, neg_epsilon, O_charge, H_charge

In [28]:
get_ion_and_water_parameters(system, topology)

(Quantity(value=1.0, unit=elementary charge),
 Quantity(value=0.2439280690268249, unit=nanometer),
 Quantity(value=0.3658460312, unit=kilojoule/mole),
 Quantity(value=-1.0, unit=elementary charge),
 Quantity(value=0.4477656957373345, unit=nanometer),
 Quantity(value=0.14891274399999999, unit=kilojoule/mole),
 Quantity(value=-0.834, unit=elementary charge),
 Quantity(value=0.417, unit=elementary charge))